# Цели исследования 

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.

В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

# Вывод первых строк всех таблиц.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [2]:
for table in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    query ='SELECT* FROM {} LIMIT 5'.format(table)
    print('Первые 5 строк таблицы {}'.format(table))
    con=engine.connect()
    display(pd.io.sql.read_sql(sql=text(query), con = con))

Первые 5 строк таблицы books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Первые 5 строк таблицы authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Первые 5 строк таблицы publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Первые 5 строк таблицы ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Первые 5 строк таблицы reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Посчитайте, сколько книг вышло после 1 января 2000 года;

In [3]:
query = '''SELECT COUNT(book_id) 
           FROM books 
           WHERE publication_date > '2000-01-01';
        '''
con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


## Для каждой книги посчитайте количество обзоров и среднюю оценку

In [4]:
query = '''SELECT b.title,
                  b.book_id,
                  COUNT (DISTINCT rw.review_id) as count_rev,
                  avg(r.rating) as av_rat
           FROM reviews AS rw
           RIGHT OUTER JOIN books AS b ON rw.book_id=b.book_id
           LEFT OUTER JOIN ratings AS r ON b.book_id = r.book_id
           GROUP BY b.book_id
           ORDER BY av_rat DESC;
        '''
con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,title,book_id,count_rev,av_rat
0,Arrows of the Queen (Heralds of Valdemar #1),86,2,5.00
1,The Walking Dead Book One (The Walking Dead #...,901,2,5.00
2,Light in August,390,2,5.00
3,Wherever You Go There You Are: Mindfulness Me...,972,2,5.00
4,Captivating: Unveiling the Mystery of a Woman'...,136,2,5.00
...,...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,915,3,2.25
996,His Excellency: George Washington,316,2,2.00
997,Drowning Ruth,202,3,2.00
998,Junky,371,2,2.00


## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [5]:
query = '''SELECT p.publisher,
                  count(b.book_id)
           FROM publishers AS p
           JOIN BOOKS AS b ON p.publisher_id=b.publisher_id
           WHERE b.num_pages > 50
           GROUP BY p.publisher_id
           ORDER BY count(b.book_id) DESC
           LIMIT 1;
        '''
con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count
0,Penguin Books,42


## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [6]:
query = '''SELECT a.author,
                  AVG(r.rating) avg_rating
           FROM authors AS a
           JOIN books AS b ON a.author_id=b.author_id
           JOIN ratings AS r ON b.book_id=r.book_id
           WHERE b.book_id IN
                (SELECT b.book_id
                 FROM books AS b
                 JOIN ratings AS r ON b.book_id=r.book_id
                 GROUP BY b.book_id
                 HAVING count(r.rating_id)>=50)
           GROUP BY a.author
           ORDER BY avg_rating desc
           LIMIT 1;
        '''
con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [8]:
query = '''WITH kob AS
          (SELECT username,
                  count(review_id)
           FROM reviews
           WHERE username in
               (SELECT username
                FROM ratings
                GROUP BY username
                HAVING count(rating_id)>48)
                GROUP BY username)
           SELECT (SUM (count) / COUNT(count)) AS AVG
           FROM kob;
        '''
con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


# Выводы по каждой задаче

В результате анализа баз данных, получены следующие данные:

1. После 1 января 2000 года вышло 819 книг.
2. Для каждой книги посчитано количество обзоров и средняя оценка.
3. Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books (42 книги).
4. Автор с самой высокой средней оценкой книг — J.K. Rowling/Mary GrandPré, с учетом книг с 50 и более оценками.
5. Среднее количество обзоров от пользователей, которые поставили больше 48 оценок — 24.33 обзора.
